In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyroomacoustics as pra
from scipy.io import wavfile
import librosa
from scipy.io import wavfile as wav
import os
import csv
import tqdm.notebook as tqdm

from numpy.random import seed
from numpy.random import randn

files = os.listdir("inputs/ESC-50-master/audio")
out_files = os.listdir("outputs/combined")

In [3]:
# read the esc50 csv - match filenames to categories
reader = csv.reader(open('inputs/esc50.csv', 'r'))
d = {}
header = True
for row in reader:
    if header == True:
        header = False
        continue
    if row[4] == "True": ## only esc10
        filename = row[0]
        category = row[3]
        d[filename] = category

In [4]:
# read the locations csv - match id to locations
reader = csv.reader(open('inputs/locations.csv', 'r'))
loc_d = {}
header = True
for row in reader:
    row = (row[0].split(" "))
    row_id = row[0]
    loc1 = np.array([float(row[1]), float(row[2]), float(row[3])])
    loc2 = np.array([float(row[4]), float(row[5]), float(row[6])])
    loc_d[row_id] = [loc1, loc2]
    


In [5]:
# The desired reverberation time and dimensions of the room
rt60_tgt = 0.3  # seconds
room_dim = [10, 10, 3]  # meters

# We invert Sabine's formula to obtain the parameters for the ISM simulator
e_absorption, max_order = pra.inverse_sabine(rt60_tgt, room_dim)

# microphone locations
mic_locs = np.c_[
    [4.9, 4, 1], 
    [5.1, 4.1, 1], 
    [5.1, 3.9, 1], 
    [5, 4, 1.2], 
    [5, 4, 0.8],
    [4.9, 3.9, 1.2],
    [5.1, 4.1, 0.8]
]

# # Create the room
# room = pra.ShoeBox(
#     room_dim, fs=fs, materials=pra.Material(e_absorption), max_order=max_order
# )

In [6]:
# fs, audio1 = wavfile.read("inputs/ESC-50-master/audio/1-137-A-32.wav")
# fs, audio2 = wavfile.read("inputs/ESC-50-master/audio/1-977-A-39.wav")

# min_len = min(audio1.shape[0], audio2.shape[0])

# audio1 = audio1[:min_len]
# audio2 = audio2[:min_len]

# print(audio1.shape)
# print(audio2.shape)

In [8]:
def simulate_sounds(file1_name, file2_name, count):
    file1_cat, file2_cat = d[file1_name], d[file2_name]
    if (file1_cat + " " + file2_cat + " " + " " + str(count) + ".wav") in out_files:
        return
    if file1_cat != file2_cat:
        fs, audio1 = wavfile.read("inputs/ESC-50-master/audio/" + file1_name)
        fs, audio2 = wavfile.read("inputs/ESC-50-master/audio/" + file2_name)

        min_len = min(audio1.shape[0], audio2.shape[0])

        audio1 = audio1[:min_len]
        audio2 = audio2[:min_len]
        
        room = pra.ShoeBox(
            room_dim, fs=fs, materials=pra.Material(e_absorption), max_order=max_order
        )
        values = np.random.random(6) * 2.0 - 1.0
        loc1 = [values[0] + 5, values[1] + 5, values[2] + 1.5]
        loc2 = [values[3] + 5, values[4] + 5, values[5] + 1.5]
        
        room.add_source(loc1, signal=audio1, delay=0.)
        room.add_source(loc2, signal=audio2, delay=0.)
        room.add_microphone_array(mic_locs)
        room.simulate()
        
        filename = file1_cat + " " + file2_cat + " " + " " + str(count) + ".wav"
        room.mic_array.to_wav(
                f"outputs/combined/" + filename,
                norm=True,
                bitdepth=np.int16,
        )

#         for loc_id in loc_d:
#             loc1 = loc_d[str(loc_id)][0]
#             loc2 = loc_d[str(loc_id)][1]
#             room = pra.ShoeBox(
#                 room_dim, fs=fs, materials=pra.Material(e_absorption), max_order=max_order
#             )
            
#             room.add_source(loc1, signal=audio1, delay=0.)
#             room.add_source(loc2, signal=audio2, delay=0.)
#             room.add_microphone_array(mic_locs)
#             room.simulate()
            
#             filename = file1_cat + " " + file2_cat + " " + loc_id + ".wav"
#             room.mic_array.to_wav(
#                 f"outputs/combined/" + filename,
#                 norm=True,
#                 bitdepth=np.int16,
#             )


In [9]:
count = 0
for i,_ in tqdm.tqdm(d.items()):
    for j,_ in tqdm.tqdm(d.items()):
        file1_name, file2_name = i, j
        simulate_sounds(file1_name, file2_name, count)
        count += 1


  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

...

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

OSError: [Errno 28] No space left on device

In [ ]:
print("HI")

In [7]:
room.add_source([0,0,0], signal=audio1, delay=0.)
room.add_source([10,10,10], signal=audio2, delay=0.)

In [9]:

room.add_microphone_array(mic_locs)

In [10]:
room.simulate()

In [11]:
room.mic_array.to_wav(
    f"outputs/combined/1.wav",
    norm=True,
    bitdepth=np.int16,
)


In [14]:
# MAKE LOCATIONS.CSV

r = list(np.arange(0., 11.0, 10.0))
h = list(np.arange(0., 4.0, 3.0))
all_r = []

for i in r:
    for j in r:
        for k in h:
            all_r.append([i, j, k])
            

count_id = 1
with open('inputs/locations.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in all_r:
        for j in all_r:
            if i != j:
                writer.writerow([count_id, i[0], i[1], i[2], j[0], j[1], j[2]])
                count_id += 1